<a href="https://colab.research.google.com/github/sofzcc/lab-m-shots/blob/main/lab-m-shots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [41]:
#!pip install --quiet openai
from openai import OpenAI
import os

#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv()) # read local .env file

#OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY_Ironhack')


# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [42]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [43]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 World Championship in 2010 driving for the Red Bull Racing team. He secured his first World Championship title at the age of 23, becoming the youngest world champion in F1 history at that time.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [44]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel
Team: Red Bull Racing


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [45]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [46]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton of the Mercedes AMG Petronas Formula One Team.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [47]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [48]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [49]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.  
Team: Renault.


Few Shots for classification.


In [50]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

### Results and Report

I'm doing the lab exercises but now using a different topic, cat food.

In [58]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''

    <style>
      pre {
          white-space: pre-wrap;
      }
    </style>
    '''))

In [59]:
# Zero-shot - the model is provided with no prior examples and must rely solely on its training data to generate responses
context_user_zero_shot = [
    {'role': 'system', 'content': 'You are an expert in cat food.'}
]
print(return_OAIResponse("What are the best ingredients for cat food?", context_user_zero_shot))


The best ingredients for cat food are ones that are highly digestible, provide a balanced nutrition profile, and are sourced from quality, whole-food sources. Some key ingredients to look for in cat food include:

1. Protein sources: Cats are obligate carnivores, so their diet should be high in animal-based protein. Look for whole meat sources like chicken, turkey, beef, and fish. Organ meats such as liver and heart can also be beneficial.

2. Healthy fats: Fats are a key energy source for cats and also aid in the absorption of fat-soluble vitamins. Look for high-quality animal fats like chicken fat or fish oil.

3. Carbohydrates: While cats do not require carbohydrates in their diet, some cat foods may contain small amounts for energy. Look for easily digestible sources such as rice or sweet potatoes.

4. Vitamins and minerals: A balanced cat food should contain essential vitamins and minerals to support overall health. Look for foods that are fortified with taurine, omega-3 fatty aci

In [68]:
# One-shot - the model is given a single example to help it understand the desired response format better
context_user_one_shot = [
    {'role': 'system', 'content':
     """You are an expert in cat food.

     What are the best ingredients for cat food?
     Ingredients: Chicken, Salmon, Tuna, Sweet Potatoes, Carrots."""}
]
print(return_OAIResponse("What are the best ingredients for cat food?", context_user_one_shot))


As an expert in cat food, I recommend choosing cat foods that contain high-quality protein sources such as chicken, salmon, and tuna. These ingredients are rich in essential nutrients that support your cat's overall health and well-being. Additionally, including ingredients like sweet potatoes and carrots can provide important vitamins, minerals, and fiber to support digestion and immunity. When selecting cat food, look for options that list these quality ingredients prominently on the label to ensure your cat is getting the nutrition they need.


In [63]:
# Few-shots - the model is given multiple examples
context_user_few_shots = [
    {'role': 'system', 'content':
     """You are an expert in cat food.

     What are the best ingredients for cat food?
     Ingredients: Chicken, Salmon, Tuna, Sweet Potatoes, Carrots.

     What ingredients should be avoided in cat food?
     Avoid: Onions, Garlic, Chocolate, Alcohol, Caffeine."""}
]
print(return_OAIResponse("What are the best ingredients for cat food?", context_user_few_shots))


The best ingredients for cat food are high-quality sources of protein such as chicken, salmon, and tuna, as well as nutrient-rich ingredients like sweet potatoes and carrots. These ingredients provide cats with essential nutrients for their overall health and well-being.


Variation 1

In [65]:
# Asking  about the best ingredients without giving context
context_basic = [
    {'role': 'system', 'content': 'You are an expert in cat food.'},
    {'role': 'user', 'content': 'What are the best ingredients for cat food?'}
]
print(return_OAIResponse("What are the best ingredients for cat food?", context_basic))


The best ingredients for cat food are those that provide a well-balanced and complete diet for your feline friend. Here are some key ingredients to look for in high-quality cat food:

1. Protein: Cats are obligate carnivores, which means they require a diet high in animal-based protein. Look for cat foods that list a high-quality source of protein, such as chicken, turkey, fish, or lamb, as the first ingredient.

2. Taurine: Taurine is an essential amino acid for cats that is crucial for their heart and eye health. Make sure the cat food you choose contains adequate levels of taurine.

3. Essential fatty acids: Omega-3 and Omega-6 fatty acids are important for maintaining healthy skin and coat, as well as supporting overall health. Look for cat foods that include sources of these essential fatty acids, such as fish oil or flaxseed.

4. Vitamins and minerals: Cats require certain vitamins and minerals to support their overall health and well-being. Choose cat foods that are fortified wi

Variation 2 - Giving options for ingredients with Coffee as an impostor ingredient.

In [64]:
context_detailed = [
    {'role': 'system', 'content':
     """You are an expert in cat food.

     What are the best ingredients for cat food?
     Ingredients: Chicken, Salmon, Tuna, Sweet Potatoes, Carrots, Spinach, Turkey, Coffee."""}
]
print(return_OAIResponse("What are the best ingredients for cat food?", context_detailed))


The best ingredients for cat food are high-quality sources of protein such as chicken, salmon, tuna, and turkey. Cats are obligate carnivores, so protein should be the primary component of their diet. Additionally, incorporating fruits and vegetables like sweet potatoes, carrots, and spinach can provide essential vitamins and minerals. It is important to avoid ingredients like coffee, as it can be toxic to cats.


Variation 3

In [66]:
context_comprehensive = [
    {'role': 'system', 'content':
     """You are an expert in cat food.

     What are the best ingredients for cat food?
     Ingredients: Chicken, Salmon, Tuna, Sweet Potatoes, Carrots, Spinach, Turkey, Coffee.

     What ingredients should be avoided in cat food?
     Avoid: Onions, Garlic, Chocolate, Alcohol, Caffeine.

     What are some good protein sources for cats?
     Protein Sources: Chicken, Turkey, Fish, Beef."""}
]
print(return_OAIResponse("What are the best ingredients for cat food?", context_comprehensive))


The best ingredients for cat food are chicken, salmon, tuna, sweet potatoes, carrots, spinach, and turkey. These ingredients are nutritious and provide cats with essential nutrients for their overall health and well-being.


Variation 4 - increased ambiguity

As the results are too close to the expected, let's add ambiguity to the prompt:

### Trying the same with a different temperature - Temperature = 0

In [69]:
# Function to call the model.
def return_OAIResponse_2(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [70]:
# Zero-shot - the model is provided with no prior examples and must rely solely on its training data to generate responses
context_user_zero_shot = [
    {'role': 'system', 'content': 'You are an expert in cat food.'}
]
print(return_OAIResponse_2("What are the best ingredients for cat food?", context_user_zero_shot))


The best ingredients for cat food are those that provide a balanced and complete nutrition for your feline friend. Here are some key ingredients to look for in cat food:

1. High-quality animal protein sources: Cats are obligate carnivores, which means they require a diet high in animal-based protein. Look for cat foods that list a high-quality protein source, such as chicken, turkey, fish, or beef, as the first ingredient.

2. Essential amino acids: Cats require certain essential amino acids, such as taurine and arginine, that are found in animal protein sources. Make sure the cat food you choose contains these essential amino acids to support your cat's overall health.

3. Healthy fats: Cats need fats in their diet for energy and to support healthy skin and coat. Look for cat foods that contain omega-3 and omega-6 fatty acids from sources like fish oil or flaxseed to promote a shiny coat and healthy skin.

4. Vitamins and minerals: Cat food should be fortified with essential vitamins

In [71]:
# One-shot - the model is given a single example to help it understand the desired response format better
context_user_one_shot = [
    {'role': 'system', 'content':
     """You are an expert in cat food.

     What are the best ingredients for cat food?
     Ingredients: Chicken, Salmon, Tuna, Sweet Potatoes, Carrots."""}
]
print(return_OAIResponse_2("What are the best ingredients for cat food?", context_user_one_shot))


The best ingredients for cat food are those that provide essential nutrients for your cat's health and well-being. Some of the top ingredients to look for in cat food include:

1. Chicken: A high-quality source of protein that helps support your cat's muscle development and overall health.

2. Salmon: Rich in omega-3 fatty acids, which are beneficial for your cat's skin, coat, and overall immune system.

3. Tuna: Another excellent source of protein and omega-3 fatty acids that can help support your cat's overall health.

4. Sweet Potatoes: A good source of carbohydrates and fiber that can help support your cat's digestion and provide energy.

5. Carrots: Rich in vitamins and minerals, carrots can help support your cat's immune system and overall health.

When choosing cat food, it's important to look for products that list these high-quality ingredients near the top of the ingredient list to ensure that your cat is getting the essential nutrients they need.


In [72]:
# Few-shots - the model is given multiple examples
context_user_few_shots = [
    {'role': 'system', 'content':
     """You are an expert in cat food.

     What are the best ingredients for cat food?
     Ingredients: Chicken, Salmon, Tuna, Sweet Potatoes, Carrots.

     What ingredients should be avoided in cat food?
     Avoid: Onions, Garlic, Chocolate, Alcohol, Caffeine."""}
]
print(return_OAIResponse_2("What are the best ingredients for cat food?", context_user_few_shots))


The best ingredients for cat food include high-quality sources of protein such as chicken, salmon, and tuna. Additionally, including nutrient-rich ingredients like sweet potatoes and carrots can provide essential vitamins and minerals for your cat's overall health and well-being.


In [73]:
# Variation 1
context_basic = [
    {'role': 'system', 'content': 'You are an expert in cat food.'},
    {'role': 'user', 'content': 'What are the best ingredients for cat food?'}
]
print(return_OAIResponse_2("What are the best ingredients for cat food?", context_basic))


The best ingredients for cat food are those that provide a balanced and complete nutrition for your feline friend. Here are some key ingredients to look for in cat food:

1. High-quality animal protein sources: Cats are obligate carnivores, which means they require a diet high in animal protein. Look for cat foods that list a high-quality animal protein source, such as chicken, turkey, fish, or beef, as the first ingredient.

2. Essential nutrients: Cats require specific nutrients such as taurine, arachidonic acid, and vitamins like A and D in their diet. Make sure the cat food you choose is formulated to meet these nutritional requirements.

3. Limited carbohydrates: Cats have a limited ability to digest carbohydrates, so it's best to choose cat foods that have a lower carbohydrate content. Look for cat foods that are grain-free or have whole grains like brown rice or oats.

4. Healthy fats: Fats are an important energy source for cats and help support their skin and coat health. Look

In [74]:
#Variation 2
context_detailed = [
    {'role': 'system', 'content':
     """You are an expert in cat food.

     What are the best ingredients for cat food?
     Ingredients: Chicken, Salmon, Tuna, Sweet Potatoes, Carrots, Spinach, Turkey, Coffee."""}
]
print(return_OAIResponse_2("What are the best ingredients for cat food?", context_detailed))


The best ingredients for cat food are high-quality sources of protein such as chicken, salmon, tuna, and turkey. These ingredients provide essential amino acids that cats need for overall health and wellbeing. Additionally, ingredients like sweet potatoes, carrots, and spinach can provide important vitamins, minerals, and fiber to support your cat's digestion and immune system. However, it is important to note that coffee is not suitable for cats and should never be included in their diet as it can be toxic to them.


In [75]:
# Variation 3
context_comprehensive = [
    {'role': 'system', 'content':
     """You are an expert in cat food.

     What are the best ingredients for cat food?
     Ingredients: Chicken, Salmon, Tuna, Sweet Potatoes, Carrots, Spinach, Turkey, Coffee.

     What ingredients should be avoided in cat food?
     Avoid: Onions, Garlic, Chocolate, Alcohol, Caffeine.

     What are some good protein sources for cats?
     Protein Sources: Chicken, Turkey, Fish, Beef."""}
]
print(return_OAIResponse_2("What are the best ingredients for cat food?", context_comprehensive))



The best ingredients for cat food are chicken, salmon, tuna, sweet potatoes, carrots, spinach, and turkey. These ingredients provide essential nutrients such as protein, vitamins, and minerals that are important for your cat's overall health and well-being.


#Report/Conclusions

**Temperature set to 1**
With zero-shot, the model provided accurate responses but din't specify formatting and details. The answers were correct but not very informative.

When given a simple example (one-shot), it improved in formatting, accuracy, and adapted well to the provided example and delivered more structured response, listing ingredients in a similar format to the example.

When given multiple examples, I got the most accurate and well-formatted outputs, with the model providing comprehensive and precise answers.

For variation 1, I tested only in a generic way by adding different ingredients and received a broad answer
However, on the variations 2 and 3 I added an impostor ingredient which is not recommended for cats so I could see how the model would handle it.

On variation 2, I received a structured response, listing ingredients in a similar format to the example. The model identified the correct ingredients and recognized that coffee is not suitable for cats, stating, "It is important to avoid ingredients like coffee, as it can be toxic to cats.", which means that the model can distinguish relevant from irrelevant or incorrect inputs.

On variation 3, themodel provided a detailed and structured response listing appropriate ingredients but ignored the impostor ingredient (Coffee). The response showed that the model could filter out incorrect information when provided with a comprehensive context.

The model didn't hallucinate when trying these examples, and proved that using M-shots significantly enhances the its ability to generate accurate and well-formatted responses. The more examples provided, the better the model performs, reducing errors and improving reliability.


**Temperature set to 0**

As the model didn't produce any hallucation, with the temperature set to 1, I changed it to 0 to see how it would affect the model.
In conclusion, when temperature is 0, the model provides consistent and focused answers, follows the example precisely, and adheres more strictly to the examples, resulting in more reliable outputs.
The variations also behave successfully, avoiding the impostor ingredient and incorrect information.
It's apparent that using a temperature of 0 significantly enhances the consistency and accuracy of the model's responses.